In [1]:
import pandas as pd
import mysql.connector

In [5]:
# Configuración de la conexión a la base de datos
db_connection = mysql.connector.connect(
    host="127.0.0.1",
    user="root",      # Cambia esto con tu usuario de MySQL
    password="11jablum11",  # Cambia esto con tu contraseña de MySQL
    database="yfinance"
)

cursor = db_connection.cursor()

In [6]:
# Cargar datos del archivo infoSP500.csv a la tabla empresas_sp500
empresas_df = pd.read_csv("../Streamlit/infoSP500.csv")

# Iterar sobre cada fila para insertar los datos en la tabla
for _, row in empresas_df.iterrows():
    cursor.execute("""
        INSERT INTO empresas_sp500 (simbolo, nombre_empresa, sector, industria)
        VALUES (%s, %s, %s, %s)
    """, (row['Symbol'], row['Security'], row['GICS Sector'], row['GICS Sub-Industry']))

# Confirmar los cambios
db_connection.commit()

# Cargar datos del archivo Cotizaciones_historicas_SP_500.csv a la tabla precios_historicos
precios_df = pd.read_csv("../Streamlit/Cotizaciones_historicas_SP_500.csv")

# Iterar sobre cada fila para insertar los datos en la tabla precios_historicos
for _, row in precios_df.iterrows():
    cursor.execute("""
        INSERT INTO precios_historicos (id_empresa, fecha, precio_apertura, precio_cierre, maximo, minimo, volumen)
        VALUES (
            (SELECT id_empresa FROM empresas_sp500 WHERE simbolo = %s), 
            %s, %s, %s, %s, %s, %s
        )
    """, (row['Symbol'], row['Date'], row['Open'], row['Close'], row['High'], row['Low'], row['Volume']))

# Confirmar los cambios
db_connection.commit()

# Cerrar la conexión a la base de datos
cursor.close()
db_connection.close()
